In [32]:

import queue
import threading
import time

import pandas
from lxml import html
from selenium.webdriver import Chrome, Edge
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def GetChromeBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    except Exception as err:
        print(err)


def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")
    options.add_argument("--mute-audio")
    options.add_argument("--disable-notifications")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)


resultQ = queue.Queue()
errorQ = queue.Queue()


def Scrapper(driver, url):
    driver.get(url)

    titleXpath = '//div[@id="centerCol"]/descendant::span[@id="productTitle"]/text()'
    priceXpath = '//div[@id="centerCol"]/descendant::span[@class="a-price-whole"]/text()'

    images = '//li[@data-csa-c-action="image-block-main-image-hover"]/descendant::img/@src'
    header = '//div[@id="productOverview_feature_div"]//*[not(self::style| self::script)]/descendant::tr/td[1]/span/text()'
    value = '//div[@id="productOverview_feature_div"]//*[not(self::style| self::script)]/descendant::tr/td[2]/span/text()'
    moreButton = "//*[contains(text(), 'About this item')]"
    moreInfo = '//div[@id="featurebullets_feature_div"]/descendant::*[not(self::style | self::script)]/text()'
    available = '//div[@id="availabilityInsideBuyBox_feature_div"]/descendant::*[not(self::style | self::script)]/text()'

    try:
        cc_element = WebDriverWait(driver, 25).until(
            EC.visibility_of_all_elements_located((By.XPATH, images)))
    except:
        pass
    d_data = {}
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * .2);")

    try:
        cc_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, moreButton)))
        cc_element.click()
    except:
        pass

    try:
        cc_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_all_elements_located((By.XPATH, titleXpath)))
    except:
        pass

    try:
        cc_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_all_elements_located((By.XPATH, priceXpath)))
    except:
        pass
    try:
        cc_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_all_elements_located((By.XPATH, moreInfo)))
    except:
        pass

    page_source = driver.page_source
    doc = html.fromstring(page_source)

    d_data['title'] = '\n'.join(doc.xpath(titleXpath))
    d_data['price'] = '\n'.join(doc.xpath(priceXpath))
    d_data['description_0'] = '\n'.join(doc.xpath(moreInfo))
    d_data['available'] = '\n'.join(doc.xpath(available))

    header = doc.xpath(header)
    values = doc.xpath(value)

    for indx, head in enumerate(header):
        d_data[head] = values[indx]

    d_data['images'] = '\n'.join(doc.xpath(images))
    d_data['product'] = url
    resultQ.put(d_data)


jobs = queue.Queue()


def WorkerLoop(progress_bar: tqdm):
    # driver = GetChromeBrowser(False)
    driver = GetEdgeBrowser(False)
    while not jobs.empty():
        url = jobs.get()
        try:
            Scrapper(driver, url)
        except:
            errorQ.put({'url': url})

            pass
        time.sleep(1)
        progress_bar.update()


i = 'amazon.csv'
path = f'amazon_files/{i}'

if i.__contains__('.csv'):
    data = pandas.read_csv(path)
else:
    data = pandas.read_excel(path)

product_column = 'a-link-normal href'
urls = list(data[product_column])

[jobs.put(_) for _ in urls]
threads = []
pbar = tqdm(total=len(urls))

for worker in range(6):
    thread = threading.Thread(target=WorkerLoop, args=(pbar,))
    thread.start()
    threads.append(thread)

for t in threads:
    t.join()


def qToDf(q):
    l = []
    while not q.empty():
        l.append(q.get())
    [q.put(_) for _ in l]
    df = pandas.DataFrame(l)
    return df


df = qToDf(resultQ)
df.to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_result.csv', index=False)
qToDf(errorQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_error.csv', index=False)




100%|██████████| 67/67 [03:09<00:00,  2.82s/it]

100%|██████████| 60/60 [03:32<00:00,  8.43s/it]